(1)
https://github.com/MiaHanzhangYuan/Lab3.git

(2)

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import psycopg2
import zipfile
import io
from sqlalchemy import create_engine
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')

In [31]:
url ='https://databank.worldbank.org/data/download/ESG_CSV.zip'
r = requests.get(url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [5]:
url = 'https://v-dem.net/media/datasets/V-Dem-CY-Core_csv_v13.zip'
r = requests.get(url)
print(r)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()


<Response [200]>


In [2]:
wb = pd.read_csv("ESGData.csv")
country = pd.read_csv("ESGCountry.csv")

In [3]:
pd.set_option('max_colwidth',10)

(c)

In [4]:
vdem = pd.read_csv("V-Dem-CY-Core-v13.csv")

In [5]:
vdem = vdem[['country_text_id', 'country_name', 'year', 'v2x_polyarchy']]

In [6]:
vdem.head(3).T

,0,1,2
country_text_id,MEX,MEX,MEX
country_name,Mexico,Mexico,Mexico
year,1789,1790,1791
v2x_polyarchy,0.028,0.028,0.028


In [7]:
vdem = vdem.query("year>=1960 & year<=2021")

In [8]:
vdem = vdem.rename({'country_text_id':'country_code',
                   'country_name':'country_name_vdem',
                   'v2x_polyarchy':'democracy'}, axis=1)

In [9]:
vdem = vdem.sort_values(by = ['country_code', 'year'], ascending = True)

(d)

In [10]:
country = country[['Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region', 'Income Group']]

In [11]:
country.columns = [x.lower() for x in country.columns]
country.columns = [x.replace(' ','_') for x in country.columns]
country.columns

Index(['country_code', 'table_name', 'long_name', 'currency_unit', 'region',
       'income_group'],
      dtype='object')

In [12]:
country = country.rename({'table_name':'country_name_wb'}, axis=1)
country

,country_code,country_name_wb,long_name,currency_unit,region,income_group
0,AFG,Afghan...,Islami...,Afghan...,South ...,Low in...
1,AGO,Angola,People...,Angola...,Sub-Sa...,Lower ...
2,ALB,Albania,Republ...,Albani...,Europe...,Upper ...
3,AND,Andorra,Princi...,Euro,Europe...,High i...
4,ARB,Arab W...,Arab W...,NaN,NaN,NaN
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan...,East A...,Lower ...
235,YEM,"Yemen,...",Republ...,Yemeni...,Middle...,Low in...
236,ZAF,South ...,Republ...,South ...,Sub-Sa...,Upper ...
237,ZMB,Zambia,Republ...,New Za...,Sub-Sa...,Lower ...


In [13]:
noncountries = ["Arab World", "Central Europe and the Baltics",
 "Caribbean small states",
 "East Asia & Pacific (excluding high income)",
 "Early-demographic dividend","East Asia & Pacific",
 "Europe & Central Asia (excluding high income)",
 "Europe & Central Asia", "Euro area",
 "European Union","Fragile and conflict affected situations",
 "High income",
 "Heavily indebted poor countries (HIPC)","IBRD only",
 "IDA & IBRD total",
 "IDA total","IDA blend","IDA only",
 "Latin America & Caribbean (excluding high income)",
 "Latin America & Caribbean",
 "Least developed countries: UN classification",
 "Low income","Lower middle income","Low & middle income",
 "Late-demographic dividend","Middle East & North Africa",
 "Middle income",
 "Middle East & North Africa (excluding high income)",
 "North America","OECD members",
 "Other small states","Pre-demographic dividend",
 "Pacific island small states",
 "Post-demographic dividend",
 "Sub-Saharan Africa (excluding high income)","Sub-Saharan Africa",
 "Small states","East Asia & Pacific (IDA & IBRD)",
 "Europe & Central Asia (IDA & IBRD)",
 "Latin America & Caribbean (IDA & IBRD)",
 "Middle East & North Africa (IDA & IBRD)","South Asia",
 "South Asia (IDA & IBRD)",
 "Sub-Saharan Africa (IDA & IBRD)",
 "Upper middle income", "World"]

In [14]:
country.query("country_name_wb not in @noncountries")

,country_code,country_name_wb,long_name,currency_unit,region,income_group
0,AFG,Afghan...,Islami...,Afghan...,South ...,Low in...
1,AGO,Angola,People...,Angola...,Sub-Sa...,Lower ...
2,ALB,Albania,Republ...,Albani...,Europe...,Upper ...
3,AND,Andorra,Princi...,Euro,Europe...,High i...
5,ARE,United...,United...,U.A.E....,Middle...,High i...
...,...,...,...,...,...,...
234,WSM,Samoa,Samoa,Samoan...,East A...,Lower ...
235,YEM,"Yemen,...",Republ...,Yemeni...,Middle...,Low in...
236,ZAF,South ...,Republ...,South ...,Sub-Sa...,Upper ...
237,ZMB,Zambia,Republ...,New Za...,Sub-Sa...,Lower ...


(e)

In [15]:
filter_col = [col for col in wb if col.startswith('19') or col.startswith('20')]

In [16]:
wb = wb[['Country Code', 'Country Name', 'Indicator Code'] + filter_col]

In [17]:
wb.head(3)

,Country Code,Country Name,Indicator Code,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ARB,Arab W...,EG.CFT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.406487,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN
1,ARB,Arab W...,EG.ELC...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.340705,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN
2,ARB,Arab W...,NY.ADJ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.598544,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN


In [18]:
wb = wb.rename({'Country Code':'country_code',
               'Country Name' : 'country_name_wb',
                'Indicator Code' : 'feature'}, axis=1)

In [19]:
noncountries.remove('World')

In [20]:
replace_map = {
 "AG.LND.AGRI.ZS": "agricultural_land",
 "AG.LND.FRST.ZS": "forest_area",
 "AG.PRD.FOOD.XD": "food_production_index",
 "CC.EST": "control_of_corruption",
 "EG.CFT.ACCS.ZS": "access_to_clean_fuels_and_technologies_for_cooking",
 "EG.EGY.PRIM.PP.KD": "energy_intensity_level_of_primary_energy",
 "EG.ELC.ACCS.ZS": "access_to_electricity",
 "EG.ELC.COAL.ZS": "electricity_production_from_coal_sources",
 "EG.ELC.RNEW.ZS": "renewable_electricity_output",
 "EG.FEC.RNEW.ZS": "renewable_energy_consumption",
 "EG.IMP.CONS.ZS": "energy_imports",
 "EG.USE.COMM.FO.ZS": "fossil_fuel_energy_consumption",
 "EG.USE.PCAP.KG.OE": "energy_use",
 "EN.ATM.CO2E.PC": "co2_emissions",
 "EN.ATM.METH.PC": "methane_emissions",
 "EN.ATM.NOXE.PC": "nitrous_oxide_emissions",
 "EN.ATM.PM25.MC.M3": "pm2_5_air_pollution",
 "EN.CLC.CDDY.XD": "cooling_degree_days",
 "EN.CLC.GHGR.MT.CE": "ghg_net_emissions",
 "EN.CLC.HEAT.XD": "heat_index_35",
 "EN.CLC.MDAT.ZS": "droughts",
 "EN.CLC.PRCP.XD": "maximum_5-day_rainfall",
 "EN.CLC.SPEI.XD": "mean_drought_index","EN.MAM.THRD.NO": "mammal_species",
 "EN.POP.DNST": "population_density",
 "ER.H2O.FWTL.ZS": "annual_freshwater_withdrawals",
 "ER.PTD.TOTL.ZS": "terrestrial_and_marine_protected_areas",
 "GB.XPD.RSDV.GD.ZS": "research_and_development_expenditure",
 "GE.EST": "government_effectiveness",
 "IC.BUS.EASE.XQ": "ease_of_doing_business_rank",
 "IC.LGL.CRED.XQ": "strength_of_legal_rights_index",
 "IP.JRN.ARTC.SC": "scientific_and_technical_journal_articles",
 "IP.PAT.RESD": "patent_applications",
 "IT.NET.USER.ZS": "individuals_using_the_internet",
 "NV.AGR.TOTL.ZS": "agriculture",
 "NY.ADJ.DFOR.GN.ZS": "net_forest_depletion",
 "NY.ADJ.DRES.GN.ZS": "natural_resources_depletion",
 "NY.GDP.MKTP.KD.ZG": "gdp_growth",
 "PV.EST": "political_stability_and_absence_of_violence",
 "RL.EST": "rule_of_law",
 "RQ.EST": "regulatory_quality",
 "SE.ADT.LITR.ZS": "literacy_rate",
 "SE.ENR.PRSC.FM.ZS": "gross_school_enrollment",
 "SE.PRM.ENRR": "primary_school_enrollment",
 "SE.XPD.TOTL.GB.ZS": "government_expenditure_on_education",
 "SG.GEN.PARL.ZS": "proportion_of_seats_held_by_women_in_national_parliamen",
 "SH.DTH.COMM.ZS": "cause_of_death",
 "SH.DYN.MORT": "mortality_rate",
 "SH.H2O.SMDW.ZS": "people_using_safely_managed_drinking_water_services",
 "SH.MED.BEDS.ZS": "hospital_beds",
 "SH.STA.OWAD.ZS": "prevalence_of_overweight",
 "SH.STA.SMSS.ZS": "people_using_safely_managed_sanitation_services",
 "SI.DST.FRST.20": "income_share_held_by_lowest_20pct",
 "SI.POV.GINI": "gini_index",
 "SI.POV.NAHC": "poverty_headcount_ratio_at_national_poverty_lines",
 "SI.SPR.PCAP.ZG": "annualized_average_growth_rate_in_per_capita_real_surve",
 "SL.TLF.0714.ZS": "children_in_employment",
 "SL.TLF.ACTI.ZS": "labor_force_participation_rate",
 "SL.TLF.CACT.FM.ZS": "ratio_of_female_to_male_labor_force_participation_ra",
 "SL.UEM.TOTL.ZS": "unemployment",
 "SM.POP.NETM": "net_migration",
 "SN.ITK.DEFC.ZS": "prevalence_of_undernourishment",
 "SP.DYN.LE00.IN": "life_expectancy_at_birth",
 "SP.DYN.TFRT.IN": "fertility_rate",
 "SP.POP.65UP.TO.ZS": "population_ages_65_and_above",
 "SP.UWT.TFRT": "unmet_need_for_contraception",
 "VA.EST": "voice_and_accountability",
 "EN.CLC.CSTP.ZS": "coastal_protection",
 "SD.ESR.PERF.XQ": "economic_and_social_rights_performance_score",
 "EN.CLC.HDDY.XD": "heating_degree_days",
 "EN.LND.LTMP.DC": "land_surface_temperature",
 "ER.H2O.FWST.ZS": "freshwater_withdrawal",
 "EN.H2O.BDYS.ZS": "water_quality",
 "AG.LND.FRLS.HA": "tree_cover_loss",
}

In [21]:
wb = wb.replace(replace_map)

In [22]:
wb

,country_code,country_name_wb,feature,1960,1961,1962,1963,1964,1965,1966,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,ARB,Arab W...,access...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.406487,86.705717,86.942778,87.228705,87.390856,87.617862,87.798740,87.948264,88.092536,NaN
1,ARB,Arab W...,access...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,89.340705,88.832276,89.053852,89.539016,90.662754,89.176939,90.352802,90.635050,90.845661,NaN
2,ARB,Arab W...,natura...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,11.598544,10.263684,6.217421,5.190735,6.458100,8.451699,7.351742,4.347213,NaN,NaN
3,ARB,Arab W...,net_fo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.062081,0.084113,0.098458,0.093562,0.095137,0.050799,0.061584,0.077949,NaN,NaN
4,ARB,Arab W...,agricu...,NaN,30.981414,30.982663,31.007054,31.018001,31.042466,31.0504,...,39.838650,39.834421,39.872575,39.937814,39.984452,39.969738,39.907031,39.973290,39.970742,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16964,ZWE,Zimbabwe,terres...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27.214542,27.214585,27.214585,27.214747,27.214747,27.214747,27.214747
16965,ZWE,Zimbabwe,tree_c...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16966,ZWE,Zimbabwe,unempl...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.994000,4.770000,5.282000,5.796000,6.279000,6.784000,7.370000,7.898000,8.067000,7.945000
16967,ZWE,Zimbabwe,unmet_...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,10.382129,10.400000,NaN,NaN,NaN,NaN,NaN,NaN,NaN


(f)

In [23]:
wb1 = pd.melt(wb, id_vars=wb.columns[:3],value_vars = wb.columns[3:])

In [24]:
wb1 = wb1.rename({'variable':'year'}, axis=1)

In [25]:
wb1 = wb1.assign(feature = wb1['feature'])
#wb1 = wb1.drop(['feature'], axis=1)

In [26]:
wb1 = wb1.pivot_table(index=['country_code','country_name_wb','year'],
                     columns = 'feature',
                     values = 'value')
wb1

feature                     access_to_clean_fuels_and_technologies_for_cooking  \
country... country... year                                                       
AFG        Afghani... 1960        NaN                                            
                      1961        NaN                                            
                      1962        NaN                                            
                      1963        NaN                                            
                      1964        NaN                                            
...                               ...                                            
ZWE        Zimbabwe   2018       30.0                                            
                      2019       30.2                                            
                      2020       30.3                                            
                      2021       30.3                                            
                      2022        NaN                                            

feature                     access_to_electricity  agricultural_land  \
country... country... year                                             
AFG        Afghani... 1960        NaN                    NaN           
                      1961        NaN              57.878356           
                      1962        NaN              57.955016           
                      1963        NaN              58.031676           
                      1964        NaN              58.116002           
...                               ...                    ...           
ZWE        Zimbabwe   2018  45.400288              41.876696           
                      2019  46.682095              41.876696           
                      2020  52.747667              41.876696           
                      2021  48.979927              41.876696           
                      2022        NaN                    NaN           

feature                     agriculture  annual_freshwater_withdrawals  \
country... country... year                                               
AFG        Afghani... 1960        NaN          NaN                       
                      1961        NaN          NaN                       
                      1962        NaN          NaN                       
                      1963        NaN          NaN                       
                      1964        NaN          NaN                       
...                               ...          ...                       
ZWE        Zimbabwe   2018   7.319375    30.761677                       
                      2019   9.819262    30.761677                       
                      2020   8.772859    30.761677                       
                      2021   8.849899          NaN                       
                      2022        NaN          NaN                       

feature                     annualized_average_growth_rate_in_per_capita_real_surve  \
country... country... year                                                            
AFG        Afghani... 1960        NaN                                                 
                      1961        NaN                                                 
                      1962        NaN                                                 
                      1963        NaN                                                 
                      1964        NaN                                                 
...                               ...                                                 
ZWE        Zimbabwe   2018        NaN                                                 
                      2019        NaN                                                 
                      2020        NaN                                                 
                      2021        NaN                                       

(g)

In [27]:
wb1 = pd.DataFrame(wb1.to_records())

In [28]:
wb1["year"] = wb1["year"].astype(int)

In [29]:
wb1

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_surve,cause_of_death,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
0,AFG,Afghan...,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghan...,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghan...,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghan...,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghan...,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15052,ZWE,Zimbabwe,2018,30.0,45.400288,41.876696,7.319375,30.761677,NaN,NaN,...,NaN,-1.292463,406.23,5.0,27.214585,NaN,6.784,NaN,-1.136798,NaN
15053,ZWE,Zimbabwe,2019,30.2,46.682095,41.876696,9.819262,30.761677,NaN,47.647301,...,NaN,-1.303515,431.62,6.0,27.214747,NaN,7.370,NaN,-1.163669,NaN
15054,ZWE,Zimbabwe,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,NaN,-1.329611,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3
15055,ZWE,Zimbabwe,2021,30.3,48.979927,41.876696,8.849899,NaN,NaN,NaN,...,NaN,-1.277202,NaN,NaN,27.214747,NaN,8.067,NaN,-1.135830,NaN


(h)

In [30]:
#Create a new dataframe containing just the rows from the World Bank country-timeseries dataset that refer to the whole world. 

wb2 = wb1[wb1['country_name_wb']=='World']

In [31]:
#Then remove the "World" rows from the World Bank country-time-series dataset

wb1 = wb1.query("country_name_wb not in ['World']")

In [32]:
#Drop the 'country_code' and 'country_name_wb' from the world dataset.

wb2 = wb2.drop(labels = ['country_code', 'country_name_wb'],axis =1)

In [33]:
#with the exception of the year column, rename all columns in the world dataset by adding "world_" in front of the column name.

wb2 = wb2.rename({'Indicator Name':'world_Indicator Name', 'Indicator Code':'world_Indicator Code'}, axis=1)

(i)

First, write a sentence stating whether you expect this merge to be one-to-one, many-to-one, one-to-many, or many-to-many, and describe your rationale.
I expect this merge to be many_to_many because I don't know about their relationship.

In [34]:
#Second, merge the two datasets together in a way that checks whether your expectation is met, and also allows you to see the rows that failed to match.
merge1 = pd.merge(wb1, vdem,
                on = ['country_code', 'year'],
                how = 'outer',
                validate = 'many_to_many',
                indicator = 'matched')

In [35]:
merge1

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_surve,cause_of_death,...,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality,country_name_vdem,democracy,matched
0,AFG,Afghan...,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.080,both
1,AFG,Afghan...,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.083,both
2,AFG,Afghan...,1962,NaN,NaN,57.955016,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.082,both
3,AFG,Afghan...,1963,NaN,NaN,58.031676,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.085,both
4,AFG,Afghan...,1964,NaN,NaN,58.116002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.137,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15384,ZZB,NaN,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zanzibar,0.267,right_...
15385,ZZB,NaN,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zanzibar,0.268,right_...
15386,ZZB,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zanzibar,0.266,right_...
15387,ZZB,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Zanzibar,0.258,right_...


(j)

In [36]:
merge1['matched'].value_counts()

matched
both          9976
left_only     5018
right_only     395
Name: count, dtype: int64

In [52]:
# First use .query() to keep only the observations that were present in the World Bank data but not V-Dem.
pd.set_option("display.max_colwidth", 70)

In [53]:
wbonly = merge1.query("matched =='left_only'")

In [54]:
#Then save the minimum and maximum values of 'year' for each country
wbmax = wbonly.groupby(['country_code', 'country_name_wb'])\
.agg({'year':'max'}).reset_index()
wbmax


,country_code,country_name_wb,year
0,AFG,Afghanistan,2022
1,AGO,Angola,2022
2,ALB,Albania,2022
3,AND,Andorra,2022
4,ARB,Arab World,2022
5,ARE,United Arab Emirates,2022
6,ARG,Argentina,2022
7,ARM,Armenia,2022
8,ATG,Antigua and Barbuda,2022
9,AUS,Australia,2022


In [55]:
wbmin = wbonly.groupby(['country_code', 'country_name_wb'])\
.agg({'year':'min'}).reset_index()
wbmin

,country_code,country_name_wb,year
0,AFG,Afghanistan,2022
1,AGO,Angola,2022
2,ALB,Albania,2022
3,AND,Andorra,1960
4,ARB,Arab World,1960
5,ARE,United Arab Emirates,1960
6,ARG,Argentina,2022
7,ARM,Armenia,1960
8,ATG,Antigua and Barbuda,1960
9,AUS,Australia,2022


In [56]:
#Then use .query() to keep only the observations that were present in the V-Dem data but not the World Bank.
vdemonly = merge1.query("matched =='right_only'")

In [57]:
# Then save the minimum and maximum values of 'year' for each country.
vdemmax = vdemonly.groupby(['country_code', 'country_name_vdem'])\
.agg({'year':'max'}).reset_index()
vdemmax

,country_code,country_name_vdem,year
0,DDR,German Democratic Republic,1990
1,HKG,Hong Kong,2021
2,PSE,Palestine/West Bank,2021
3,PSG,Palestine/Gaza,2021
4,SML,Somaliland,2021
5,TWN,Taiwan,2021
6,VDR,Republic of Vietnam,1975
7,XKX,Kosovo,2021
8,YMD,South Yemen,1990
9,ZZB,Zanzibar,2021


In [58]:
vdemmin = vdemonly.groupby(['country_code', 'country_name_vdem'])\
.agg({'year':'min'}).reset_index()
vdemmin

,country_code,country_name_vdem,year
0,DDR,German Democratic Republic,1960
1,HKG,Hong Kong,1960
2,PSE,Palestine/West Bank,1967
3,PSG,Palestine/Gaza,1960
4,SML,Somaliland,1991
5,TWN,Taiwan,1960
6,VDR,Republic of Vietnam,1960
7,XKX,Kosovo,1999
8,YMD,South Yemen,1960
9,ZZB,Zanzibar,1960


(k)

* DEU in world bank data is germany; DDR in vdem data is german Democratic Republic; I think they might be the same thing.
* The VDEM dataset includes districts that are not officially recognized as independent countries.
* Each organization or research project may have its own specific criteria and methodology for data collection. Differences in the data collection process, including the selection of countries, might lead to variations in country coverage.

(l)

In [61]:
#repeat the merge, this time dropping all unmatched observations. This time there is no need to validate the type of merge, and no need to define a variable to indicate matching.
pd.set_option('display.max_row', 4)
merge2 = pd.merge(wb1, vdem, on = ['country_code', 'year'])
merge2

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_surve,cause_of_death,...,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality,country_name_vdem,democracy
0,AFG,Afghan...,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.080
1,AFG,Afghan...,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9974,ZWE,Zimbabwe,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3,Zimbabwe,0.293
9975,ZWE,Zimbabwe,2021,30.3,48.979927,41.876696,8.849899,NaN,NaN,NaN,...,NaN,NaN,27.214747,NaN,8.067,NaN,-1.135830,NaN,Zimbabwe,0.291


# (3)

In [62]:
world = wb2

In [63]:
world

,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_surve,cause_of_death,children_in_employment,co2_emissions,...,research_and_development_expenditure,rule_of_law,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality
14679,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14680,1961,NaN,NaN,35.879317,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14740,2021,71.331487,91.413941,36.841665,4.304093,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,14.55226,NaN,6.201871,NaN,NaN,NaN
14741,2022,NaN,NaN,NaN,4.329327,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,5.772710,NaN,NaN,NaN


In [64]:
timeseries = merge2

In [65]:
timeseries

,country_code,country_name_wb,year,access_to_clean_fuels_and_technologies_for_cooking,access_to_electricity,agricultural_land,agriculture,annual_freshwater_withdrawals,annualized_average_growth_rate_in_per_capita_real_surve,cause_of_death,...,scientific_and_technical_journal_articles,strength_of_legal_rights_index,terrestrial_and_marine_protected_areas,tree_cover_loss,unemployment,unmet_need_for_contraception,voice_and_accountability,water_quality,country_name_vdem,democracy
0,AFG,Afghan...,1960,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.080
1,AFG,Afghan...,1961,NaN,NaN,57.878356,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Afghan...,0.083
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9974,ZWE,Zimbabwe,2020,30.3,52.747667,41.876696,8.772859,30.761677,NaN,NaN,...,480.16,NaN,27.214747,NaN,7.898,NaN,-1.113408,83.3,Zimbabwe,0.293
9975,ZWE,Zimbabwe,2021,30.3,48.979927,41.876696,8.849899,NaN,NaN,NaN,...,NaN,NaN,27.214747,NaN,8.067,NaN,-1.135830,NaN,Zimbabwe,0.291


1. First NF: Every table must have a primary key; The values inside every cell in every table must be atomic; There are no repeating groups.
   * In the timeseries table there's two same columns so we can drop one.
   

In [66]:
timeseries = timeseries.drop('country_name_vdem', axis = 1)

2. Second NF: All the non-prime attributes need to depend on ALL of the attributes within the primary key.
 * In timeseries table, the "country_name_wb" only depends on the "country_code" doesn't include "year". So we make a new table and drop this column.

In [67]:
country.head(3).T

,0,1,2
country_code,AFG,AGO,ALB
country_name_wb,Afghan...,Angola,Albania
...,...,...,...
region,South ...,Sub-Sa...,Europe...
income_group,Low in...,Lower ...,Upper ...


In [68]:
timeseries = timeseries.drop('country_name_wb', axis = 1)

4. Third NF: No non-prime attribute has a functional dependency on another non-prime attribute.
   * Three tables follows 3rd NF.

# (4)

In [69]:
dbserver = psycopg2.connect(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres'
)
dbserver.autocommit = True

In [70]:
cursor = dbserver.cursor()

In [71]:
try:
    cursor.execute('CREATE DATABASE cardib')
except:
    cursor.execute('DROP DATABASE cardib')
    cursor.execute('CREATE DATABASE cardib')

In [72]:
engine = create_engine("postgresql+psycopg2://{user}:{pw}@{service}/{db}".format(
    user = 'postgres',
    pw = POSTGRES_PASSWORD,
    service = 'postgres',
    db = 'cardib'
))

In [73]:
world.to_sql('world', con=engine, index=False, chunksize=1000, if_exists='replace')

63

In [74]:
country.to_sql('country', con=engine, index=False, chunksize=1000, if_exists='replace')

239

In [75]:
timeseries.to_sql('timeseries', con=engine, index=False, chunksize=1000, if_exists='replace')

1156

# (5)

https://dbdocs.io/yuanhanzhang01/CARDIB

# (6)

In [76]:
# What countries had the highest quality democracies in the year 2021? Join the timeseries and country tables, select the country_name_wb and democracy attributes, rename county_name_wb to country, filter the rows to only those for the year 2021, and sort the rows in descending order by the value of democracy.

myquery = '''
SELECT
    c.country_name_wb AS country,
    t.democracy
FROM
    timeseries t
JOIN
    country c ON t.country_code = c.country_code
WHERE
    t.year = 2021
ORDER BY
    t.democracy DESC;
'''

pd.read_sql_query(myquery, con=engine)

,country,democracy
0,Denmark,0.915
1,Sweden,0.903
...,...,...
170,Eritrea,0.072
171,Saudi ...,0.016


In [78]:
myquery = '''
SELECT 
    t.year,
    t.life_expectancy_at_birth AS CHL_life_expectancy,
    w.life_expectancy_at_birth AS world_life_expectancy
FROM 
    timeseries t
JOIN 
    world w ON t.year = w.year
WHERE 
    t.country_code = 'CHL'
ORDER BY 
    t.year;
'''

pd.read_sql_query(myquery, con=engine)

,year,chl_life_expectancy,world_life_expectancy
0,1960,57.015,50.894180
1,1961,57.537,52.846336
...,...,...,...
60,2020,79.377,72.243822
61,2021,NaN,NaN


In [79]:
myquery = '''
SELECT country.region, SUM(timeseries.co2_emissions) AS co2_emissions
FROM timeseries
JOIN country ON timeseries.country_code = country.country_code
WHERE timeseries.year = 2019
GROUP BY country.region
ORDER BY co2_emissions DESC;

'''

pd.read_sql_query(myquery, con=engine)


,region,co2_emissions
0,Europe...,273.80...
1,Middle...,176.00...
...,...,...
5,North ...,29.726128
6,South ...,10.820011


In [80]:
myquery = '''
SELECT
    c.country_name_wb,
    t1.democracy AS democracy_1960,
    t2.democracy AS democracy_2021,
    t2.democracy - t1.democracy AS democracy_difference
FROM
    country c
JOIN
    (SELECT country_code, democracy FROM timeseries WHERE year = 1960) t1
ON
    c.country_code = t1.country_code
JOIN
    (SELECT country_code, democracy FROM timeseries WHERE year = 2021) t2
ON
    c.country_code = t2.country_code
ORDER BY
    democracy_difference DESC
LIMIT 10;


'''

pd.read_sql_query(myquery, con=engine)



,country_name_wb,democracy_1960,democracy_2021,democracy_difference
0,Spain,0.070,0.854,0.784
1,Portugal,0.128,0.888,0.760
...,...,...,...,...
8,Seyche...,0.130,0.714,0.584
9,Malawi,0.040,0.619,0.579


In [81]:
myquery = '''
SELECT
    currency_unit,
    COUNT(*) AS country_count
FROM
    country
GROUP BY
    currency_unit
ORDER BY
    country_count DESC
LIMIT 2;
'''

pd.read_sql_query(myquery, con=engine)

,currency_unit,country_count
0,None,46
1,Euro,24


In [82]:
myquery = '''
SELECT
    c.income_group,
    AVG(t.gini_index) AS average_gini_index
FROM
    country c
JOIN
    timeseries t ON c.country_code = t.country_code
WHERE
    t.year = 2019
GROUP BY
    c.income_group
ORDER BY
    average_gini_index;

'''

pd.read_sql_query(myquery, con=engine)

,income_group,average_gini_index
0,High i...,31.73125
1,Lower ...,37.42000
...,...,...
3,Low in...,43.90000
4,None,NaN
